In [14]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools
from datetime import datetime
import csv
from os import listdir

In [15]:
seasonData = pd.read_csv('LeagueData/Pre20202021.csv', error_bad_lines=False)
display(seasonData.head())

# Initial dataset used for each season 

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD
0,1,12/09/2020,Fulham,Arsenal,0,3,A,0,1,A,...,3,2,2,0,0,6.00,4.33,1.53,1.53,4.0
1,2,12/09/2020,Crystal Palace,Southampton,1,0,H,1,0,H,...,3,2,1,0,0,3.10,3.25,2.37,1.90,3.4
2,3,12/09/2020,Liverpool,Leeds,4,3,H,3,2,H,...,0,1,0,0,0,1.28,6.00,9.50,2.45,3.3
3,4,12/09/2020,West Ham,Newcastle,0,2,A,0,0,D,...,7,2,2,0,0,2.15,3.40,3.40,6.25,3.9
4,5,13/09/2020,West Brom,Leicester,0,3,A,0,0,D,...,5,1,1,0,0,3.80,3.60,1.95,3.80,3.5


In [16]:
# Columns in dataset used
cols = ['HomeTeam','AwayTeam','FTHG','FTAG','FTR','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR','B365H','B365D','B365A']
SeasonData = seasonData[cols]
SeasonData

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A
0,Fulham,Arsenal,0,3,A,5,13,2,6,12,12,2,3,2,2,0,0,6.00,4.33,1.53
1,Crystal Palace,Southampton,1,0,H,5,9,3,5,14,11,7,3,2,1,0,0,3.10,3.25,2.37
2,Liverpool,Leeds,4,3,H,22,6,6,3,9,6,9,0,1,0,0,0,1.28,6.00,9.50
3,West Ham,Newcastle,0,2,A,15,15,3,2,13,7,8,7,2,2,0,0,2.15,3.40,3.40
4,West Brom,Leicester,0,3,A,7,13,1,7,12,9,2,5,1,1,0,0,3.80,3.60,1.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Liverpool,Crystal Palace,3,0,H,25,9,10,3,13,8,3,3,0,1,0,0,1.14,9.00,23.00
376,Man City,Everton,2,0,H,9,6,2,1,16,10,3,6,2,0,0,0,2.50,3.30,3.10
377,Sheffield,Burnley,0,1,A,14,14,6,3,12,10,4,10,2,4,0,0,1.80,3.80,4.75
378,West Ham,Southampton,2,1,H,12,16,2,5,8,10,7,4,1,1,0,0,2.10,3.50,3.75


In [17]:
def goalsScored(goalStat):
    #All scores are added to a column as a features
    teams = {}
    for i in goalStat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    for i in range(len(goalStat)):
        HTGS = goalStat.iloc[i]['FTHG']
        ATGS = goalStat.iloc[i]['FTAG']
        teams[goalStat.iloc[i].HomeTeam].append(HTGS)
        teams[goalStat.iloc[i].AwayTeam].append(ATGS)
    
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsScored[0] = 0
    for i in range(2,39):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored



def goalsConceded(goalStat):
    #All goals conceded are added to a column as a features
    teams = {}
    for i in goalStat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    for i in range(len(goalStat)):
        ATGC = goalStat.iloc[i]['FTHG']
        HTGC = goalStat.iloc[i]['FTAG']
        teams[goalStat.iloc[i].HomeTeam].append(HTGC)
        teams[goalStat.iloc[i].AwayTeam].append(ATGC)
    
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsConceded[0] = 0
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded

def HAGSGC(goalStat):
    # Uses the previous two functions to add the HTGC, ATGC, HTGS and HTGS
    #Home Team / Away Team. Goals Scored/ Goals Conceded
    GC = goalsConceded(goalStat)
    GS = goalsScored(goalStat)
   
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    for i in range(380):
        ht = goalStat.iloc[i].HomeTeam
        at = goalStat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% 10) == 0:
            j = j + 1
        
    goalStat['HTGS'] = HTGS
    goalStat['ATGS'] = ATGS
    goalStat['HTGC'] = HTGC
    goalStat['ATGC'] = ATGC
    
    return goalStat


# Calls the function
SeasonData = HAGSGC(SeasonData)

<ipython-input-17-3545bcf848c3>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playing_stat['HTGS'] = HTGS
<ipython-input-17-3545bcf848c3>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playing_stat['ATGS'] = ATGS
<ipython-input-17-3545bcf848c3>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [18]:
def getPoints(result):
    # Points increase by 3 for a win and by 1 for a draw
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    

def CumulPoints(matches):
    # Adds the points cumulatively whenever a team attains points for each week
    matchesPoints = matches.applymap(getPoints)
    # 38 fixtures represent the 38 games in a season for each team
    for i in range(2,39):
        matchesPoints[i] = matchesPoints[i] + matchesPoints[i-1]
    
    #20 columns represent the 20 teams 
    matchesPoints.insert(column =0, loc = 0, value = [0*i for i in range(20)])
    return matchesPoints


def getMatch(goalStat):
    # Create a dictionary for each team storing the total number of Wins, Draws and Losses
    teams = {}
    for i in goalStat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    for i in range(len(goalStat)):
        if goalStat.iloc[i].FTR == 'H':
            teams[goalStat.iloc[i].HomeTeam].append('W')
            teams[goalStat.iloc[i].AwayTeam].append('L')
        elif goalStat.iloc[i].FTR == 'A':
            teams[goalStat.iloc[i].AwayTeam].append('W')
            teams[goalStat.iloc[i].HomeTeam].append('L')
        else:
            teams[goalStat.iloc[i].AwayTeam].append('D')
            teams[goalStat.iloc[i].HomeTeam].append('D')
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

def aggrePoints(goalStat):
    # Uses the previous functions to calculate the points each team achieves at each game week for the entire season
    matches = getMatch(goalStat)
    cumulPts = CumulPoints(matches)
    HTP = []
    ATP = []
    j = 0
    for i in range(380):
        ht = goalStat.iloc[i].HomeTeam
        at = goalStat.iloc[i].AwayTeam
        HTP.append(cumulPts.loc[ht][j])
        ATP.append(cumulPts.loc[at][j])

        if ((i + 1)% 10) == 0:
            j = j + 1
            
    goalStat['HTP'] = HTP
    goalStat['ATP'] = ATP
    return goalStat
    
# Adds each teams points to the dataset for both the Home Team and Away Team cumulatively by changing each gameweek
SeasonData = aggrePoints(SeasonData)

<ipython-input-18-11d8f73b8488>:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playing_stat['HTP'] = HTP
<ipython-input-18-11d8f73b8488>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playing_stat['ATP'] = ATP


In [19]:
def Findform(goalStat,num):
    # Finds the team form by seeing the result of the previous fixture and returning to addForm
    form = getMatch(goalStat)
    formFinal = form.copy()
    for i in range(num,39):
        formFinal[i] = ''
        j = 0
        while j < num:
            formFinal[i] += form[i-j]
            j += 1           
    return formFinal

def addForm(goalStat,num):
    form = Findform(goalStat,num)
    h = ['M' for i in range(num * 10)]
    a = ['M' for i in range(num * 10)]
    
    j = num
    #For each fixture the result of the previous fixture is added
    for i in range((num*10),380):
        ht = goalStat.iloc[i].HomeTeam
        at = goalStat.iloc[i].AwayTeam
        
        past = form.loc[ht][j]
        h.append(past[num-1])         
        
        past = form.loc[at][j] 
        a.append(past[num-1])     
        
        if ((i + 1)% 10) == 0:
            j = j + 1

    goalStat['HM' + str(num)] = h                 
    goalStat['AM' + str(num)] = a
    return goalStat


def addForm1(Statistics):
    Statistics = addForm(Statistics,1)
    return Statistics    
    
# Adds the teams form for the previous fixture
SeasonData = addForm1(SeasonData)

<ipython-input-19-e28ebfe8aa8a>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playing_stat['HM' + str(num)] = h


In [24]:
# League table for last season with positions
# Recently promoted teams get the values 21,22 and 23 based on there position in the Championship
cols = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR','B365H','B365D','B365A']

SeasonData = SeasonData[cols]
Standings = pd.read_csv("EPLStandings.csv")
Standings.set_index(['Team'], inplace=True)

def lastResult(goalStat, Standings, year):
    HomeTeamLP = []
    AwayTeamLP = []
    for i in range(380):
        ht = goalStat.iloc[i].HomeTeam
        at = goalStat.iloc[i].AwayTeam
        HomeTeamLP.append(Standings.loc[ht][year])
        AwayTeamLP.append(Standings.loc[at][year])
    goalStat['HomeTeamLP'] = HomeTeamLP
    goalStat['AwayTeamLP'] = AwayTeamLP
    return goalStat

# Finds the final league position for the last season by using a CSV files data with the 2019/2020 season table
SeasonData = lastResult(SeasonData, Standings, 0)

In [25]:
#Function shows the match week every 10 games the match week iterates to display a new gameweek
def getmatchweek(goalStat):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat['MW'] = MatchWeek
    return playing_stat

playing_statistics_1 = getmatchweek(playing_statistics_1)

In [26]:
# Adds the teams form for the previous fixture by finding the result for each team
goalStat = pd.concat([SeasonData], ignore_index=True)
def formPoints(string):
    sum = 0
    for resultletter in string:
        sum += getPoints(resultletter)
    return sum

goalStat['HomeFormPtsStr'] = goalStat['HM1']
goalStat['HomeFormPts'] = goalStat['HomeFormPtsStr'].apply(formPoints)
goalStat.keys()

Index(['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC',
       'ATGC', 'HTP', 'ATP', 'HM1', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'HomeTeamLP',
       'AwayTeamLP', 'HomeFormPtsStr', 'HomeFormPts'],
      dtype='object')

In [28]:
goalStat.to_csv("Pre2_2021.csv")
# Creates a dataset with all the added features